In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.5e6
path = 'result/transferRL_' + str(round(n_site)) + 'patch'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2500000.00, reward:-229.27
 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2500000.00, reward:-229.27
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2500000.00, reward:-554.35
 Time: 1 | S: 25495797.0, E: 144.0, A: 0.0, I: 47.3, D: 0.0, R: 4.0, Budget: 2500000.00, reward:-554.35
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

[0.1602024168782936, 0.1607846639232789]
[0.8114582746134343, 0.8137392403147089]
[0.09728105371234036, 0.0973071796205921]
[0.043434609770821295, 0.04352760694406809]
 Time: 10 | S: 25430005.8, E: 151.8, A: 1193.5, I: 1719.2, D: 3.6, R: 62926.3, Budget: 2390674.94, reward:-10078.95
 Time: 10 | S: 25430221.0, E: 148.0, A: 1193.5, I: 1719.2, D: 3.6, R: 62912.0, Budget: 2390674.94, reward:-10078.95
 Time: 10 | new_e:71.4, cum_e:2267.4, new_i:191.7, cum_i:2331.8
[[2.3538345e-07 1.1620095e-06 6.7029094e-07 9.5160753e-01 3.9013583e-05
  9.9666852e-01 2.6731383e-08 1.3680848e-07 1.5701732e-03 8.6148000e-01
  9.9999869e-01 9.9999630e-01 9.9903405e-01 1.6898654e-01 8.8200572e-08
  9.9999857e-01]
 [9.9999833e-01 1.0000000e+00 9.9998057e-01 9.9999416e-01 9.9999642e-01
  9.9999952e-01 9.9999845e-01 1.5096734e-05 9.9999762e-01 9.9999940e-01
  9.9999976e-01 4.7022269e-08 1.0000000e+00 9.9872273e-01 3.3461448e-02
  6.9823368e-06]
 [1.0000000e+00 9.9999952e-01 1.0000000e+00 1.0000000e+00 1.0000000e+0

[0.16041190850151152, 0.16041500314001805]
[0.8136843473957289, 0.8136862668259482]
[0.09730448544073482, 0.0973044867384106]
[0.0435226500103822, 0.04352265388504649]
 Time: 15 | S: 25312214.9, E: 10.2, A: 674.9, I: 1601.6, D: 6.9, R: 181491.5, Budget: 2201954.50, reward:-10657.85
 Time: 15 | S: 25312417.0, E: 16.0, A: 674.9, I: 1601.6, D: 6.9, R: 181470.0, Budget: 2201954.50, reward:-10657.85
 Time: 15 | new_e:4.9, cum_e:2356.3, new_i:76.6, cum_i:2892.9
[[2.6658878e-02 1.0235088e-04 9.4867154e-09 5.4675631e-02 9.9798042e-01
  9.3652105e-01 2.8031353e-08 1.4786482e-08 5.9741281e-04 1.4731671e-01
  1.0000000e+00 9.9999559e-01 9.9750286e-01 4.3400913e-05 2.8706774e-08
  9.9887270e-01]
 [1.0000000e+00 1.0000000e+00 9.9999118e-01 9.9999487e-01 1.0000000e+00
  9.9999928e-01 9.9986613e-01 9.9957126e-01 9.9987924e-01 1.0000000e+00
  1.0000000e+00 2.3216202e-09 9.9999976e-01 1.6255273e-02 9.9714869e-01
  2.8990533e-08]
 [9.9999797e-01 9.9999964e-01 9.9998450e-01 1.0000000e+00 9.9999905e-01
  

[0.1604135035296509, 0.16041350842392757]
[0.8136860517244645, 0.8136861784149625]
[0.0973044865824492, 0.09730448658251628]
[0.04352265353218918, 0.043522653532278514]
 Time: 21 | S: 25146903.9, E: 0.1, A: 258.4, I: 1137.9, D: 10.1, R: 347689.5, Budget: 1935338.88, reward:-10877.91
 Time: 21 | S: 25147094.0, E: 0.0, A: 258.4, I: 1137.9, D: 10.1, R: 347667.0, Budget: 1935338.88, reward:-10877.91
 Time: 21 | new_e:0.1, cum_e:2360.8, new_i:27.7, cum_i:3156.3
[[8.5269562e-11 1.9291866e-08 2.2905472e-07 3.0986099e-03 6.0660653e-07
  3.3979729e-06 9.8604023e-01 1.8507800e-06 9.9871671e-01 6.3292903e-04
  9.9997485e-01 1.0000000e+00 1.0000000e+00 3.9677092e-05 2.2046227e-06
  9.9994218e-01]
 [1.0000000e+00 1.0000000e+00 3.9825219e-04 9.9989867e-01 9.9999452e-01
  9.9999738e-01 9.9957639e-01 5.7217710e-02 1.0000000e+00 1.0000000e+00
  1.0000000e+00 2.8247655e-07 1.0000000e+00 8.1509534e-05 2.6918657e-05
  9.9884427e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 

[0.16041350570493312, 0.16041350572542498]
[0.8136861678285093, 0.8136861688534166]
[0.09730448658251069, 0.09730448658251073]
[0.04352265353227614, 0.04352265353227626]
 Time: 27 | S: 25056369.6, E: 0.0, A: 95.8, I: 844.8, D: 13.7, R: 438676.2, Budget: 1686044.31, reward:-11081.64
 Time: 27 | S: 25056555.0, E: 0.0, A: 95.8, I: 844.8, D: 13.7, R: 438654.0, Budget: 1686044.31, reward:-11081.64
 Time: 27 | new_e:0.0, cum_e:2360.8, new_i:10.3, cum_i:3253.2
[[2.8336675e-09 1.1959808e-07 5.9591883e-09 3.8790178e-05 2.0900002e-06
  1.4717857e-05 6.9745292e-04 3.5933294e-08 9.7884709e-01 4.8662911e-04
  9.9999964e-01 1.0000000e+00 9.9978191e-01 2.2391341e-05 8.5032212e-07
  9.9997902e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999034e-01 9.9999988e-01
  9.9999964e-01 9.9999988e-01 9.9964535e-01 1.0000000e+00 1.0000000e+00
  9.9999905e-01 1.2853224e-10 1.0000000e+00 9.3930238e-04 3.2046166e-05
  9.9992335e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.

...

[0.16041350572380392, 0.16041350572380397]
[0.8136861685715336, 0.8136861685715474]
[0.09730448658251073, 0.09730448658251073]
[0.043522653532276245, 0.043522653532276245]
 Time: 60 | S: 24660780.6, E: 0.0, A: 0.4, I: 32.8, D: 18.2, R: 835168.1, Budget: 535171.63, reward:-11320.92
 Time: 60 | S: 24660962.0, E: 0.0, A: 0.4, I: 32.8, D: 18.2, R: 835150.0, Budget: 535171.63, reward:-11320.92
 Time: 60 | new_e:0.0, cum_e:2360.9, new_i:0.0, cum_i:3310.0
[[2.8087372e-09 2.3615446e-07 1.5504033e-02 3.5970182e-08 7.0122878e-05
  1.6395028e-10 9.3905275e-09 1.4851865e-04 7.5294382e-10 7.4895583e-11
  9.8211449e-01 1.0000000e+00 9.9999034e-01 8.8284945e-01 8.8660585e-05
  4.4211060e-07]
 [1.0000000e+00 1.0000000e+00 9.9999988e-01 9.9999690e-01 9.9999714e-01
  1.0000000e+00 1.0000000e+00 9.9708253e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01
  6.1078178e-09]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.00000

[0.16041350572380392, 0.16041350572380397]
[0.8136861685715382, 0.8136861685715447]
[0.09730448658251073, 0.09730448658251073]
[0.043522653532276245, 0.043522653532276245]
 Time: 66 | S: 24631700.9, E: 0.0, A: 0.2, I: 17.9, D: 18.2, R: 864262.8, Budget: 370336.76, reward:-11323.91
 Time: 66 | S: 24631880.0, E: 0.0, A: 0.2, I: 17.9, D: 18.2, R: 864245.0, Budget: 370336.76, reward:-11323.91
 Time: 66 | new_e:0.0, cum_e:2360.9, new_i:0.0, cum_i:3310.2
[[9.2253707e-09 5.3612794e-08 1.8856442e-02 3.8307235e-02 1.0623780e-01
  7.1068229e-07 5.3874318e-11 4.9628766e-06 2.5656814e-12 5.9848027e-10
  9.9982750e-01 9.9999976e-01 2.5541552e-03 9.9995708e-01 1.7790976e-03
  1.0612039e-08]
 [1.0000000e+00 1.0000000e+00 9.9999988e-01 9.9999595e-01 9.9999928e-01
  9.9999905e-01 1.0000000e+00 8.1862086e-01 1.0000000e+00 1.0000000e+00
  9.9999988e-01 1.0000000e+00 1.0000000e+00 9.9999976e-01 1.0000000e+00
  3.6868437e-09]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.00000

[0.16041350572380392, 0.16041350572380397]
[0.8136861685715403, 0.8136861685715434]
[0.09730448658251073, 0.09730448658251073]
[0.043522653532276245, 0.043522653532276245]
 Time: 72 | S: 24590548.8, E: 0.0, A: 0.1, I: 11.1, D: 18.3, R: 905421.8, Budget: 219924.45, reward:-11326.44
 Time: 72 | S: 24590724.0, E: 0.0, A: 0.1, I: 11.1, D: 18.3, R: 905405.0, Budget: 219924.45, reward:-11326.44
 Time: 72 | new_e:0.0, cum_e:2360.9, new_i:0.0, cum_i:3310.2
[[1.8755100e-13 7.3590883e-10 3.7273785e-06 3.2839102e-06 4.7741029e-03
  2.0189624e-02 2.3337583e-09 2.5568102e-08 3.6698541e-01 7.9671061e-11
  9.9986970e-01 1.0000000e+00 9.9998343e-01 9.9999738e-01 1.3013192e-06
  6.0625666e-08]
 [1.0000000e+00 1.0000000e+00 5.0515521e-01 9.9999988e-01 9.9999893e-01
  9.9999964e-01 1.0000000e+00 9.9999547e-01 1.0000000e+00 9.9999797e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999821e-01
  2.8256977e-06]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.00000

[0.16041350572380392, 0.16041350572380397]
[0.8136861685715413, 0.8136861685715427]
[0.09730448658251073, 0.09730448658251073]
[0.043522653532276245, 0.043522653532276245]
 Time: 78 | S: 24574421.3, E: 0.0, A: 0.0, I: 8.1, D: 18.3, R: 921552.3, Budget: 56656.20, reward:-11328.86
 Time: 78 | S: 24574594.0, E: 0.0, A: 0.0, I: 8.1, D: 18.3, R: 921536.0, Budget: 56656.20, reward:-11328.86
 Time: 78 | new_e:0.0, cum_e:2360.9, new_i:0.0, cum_i:3310.3
[[3.4660656e-13 4.1763013e-07 1.1755157e-05 3.2247509e-09 4.2259948e-05
  2.4813249e-11 1.3673168e-06 3.7232958e-08 4.3764040e-10 2.2235274e-12
  9.7459996e-01 1.0000000e+00 9.9874347e-01 9.9240148e-01 1.1971421e-08
  5.6392228e-04]
 [1.0000000e+00 9.9999845e-01 1.0000000e+00 1.0000000e+00 9.9838853e-01
  1.0000000e+00 1.0000000e+00 2.3898847e-01 1.0000000e+00 9.9996006e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  3.4124972e-10]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+

 Time: 84 | S: 24594574.7, E: 7.1, A: 0.0, I: 14.6, D: 18.4, R: 901385.2, Budget: 28465.34, reward:-11389.95
 Time: 84 | S: 24594752.0, E: 0.0, A: 0.0, I: 14.6, D: 18.4, R: 901367.0, Budget: 28465.34, reward:-11389.95
 Time: 84 | new_e:4.5, cum_e:2374.1, new_i:3.8, cum_i:3320.9
[[2.5525077e-08 4.0386221e-05 9.9999094e-01 1.6659079e-07 3.7378699e-03
  4.7581809e-05 6.1346800e-10 8.5379472e-03 2.8441399e-03 9.3645036e-07
  1.3707956e-08 9.9327374e-01 9.9999321e-01 7.3946330e-06 3.2352464e-06
  2.5339510e-05]
 [1.0000000e+00 9.9999940e-01 9.9689865e-01 9.9438238e-01 9.9992216e-01
  9.8755193e-01 9.9999893e-01 7.7653222e-06 9.9999988e-01 9.9999738e-01
  9.9999893e-01 9.9999976e-01 9.9999976e-01 1.0000000e+00 1.8058369e-02
  2.6557359e-07]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00
  9.9981517e-01 9.9999964e-01 9.9999940e-01 9.9999988e-01 9.9999976e-01
  1.0000000e+00 1.0000000e+00 9.9999952e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [9.9999988e-01 9.9979

In [6]:
(time.time()-tic) / ( T - start_time + 1)

61.08016003440408